# Obtaining the starting point for *yambo*: *QuantumEspresso* self and non self-consistent simulations.

***Prerequisites***: we suppose that you have successfully installed and configured AiiDA, aiida-quantuespresso and you have stored computer, codes, structures and pseudo in the AiiDA database.

In the following we show how to easily run a QE scf+nscf flow, in order to have a starting ground state density and wavefunctions needed to perform GW and BSE simulations. For more details, please have a look at the [AiiDA tutorials](https://aiida-tutorials.readthedocs.io/en/latest/). 
Quantum simulations here are performed by means of the *QuantumEspresso* simulation package, in particular using the ***pw.x*** executable. Pseudpotentials employed will be the [PseudoDojo](http://www.pseudo-dojo.org/) ones.

In [4]:
from aiida import orm, load_profile
load_profile()

Profile<uuid='b35700dae723411ea16ebc82d58f16bc' name='mb'>

## (1) Self-consistent field simulation.

This is step is dedicated to obtain the ground state density by means of a self-consistent simulation. This is performed by means of the `PwBaseWorkChain` of the ***aiida-quantumespresso*** plugin. This workflow provides automatic error handling and automatic input creation (by means of pre-defined protocols for default parameters). 
For more details in the ***aiida-quantumespresso*** plugin you can have a look at the corresponding [documentation](https://aiida-quantumespresso.readthedocs.io/en/latest/).

In [7]:
from aiida.plugins import WorkflowFactory
PwBaseWorkchain = WorkflowFactory('quantumespresso.pw.base')

You can try also to load the `PwRelaxWorkchain` class, which is used to perform structural relaxation. The corresponding entry-point is `quantumespresso.pw.relax`. 
Here, we that our input structure is already relaxed.

In the following we create the *builder* instance, which essentially collects all the inputs needed in our `PwBaseWorkchain` to be executed. We will use the `get_builder_from_protocol` methods to pre-populate the builder with default parameters.

In [10]:
from aiida_quantumespresso.common.types import ElectronicType

builder = PwBaseWorkchain.get_builder_from_protocol(
                code=orm.load_code('pw-7.1@hydralogin'),
                structure=orm.load_node(161),              # <=== this is the structure store before, in 01_structure_and_pseudos.ipynb
                protocol='fast',
                overrides={},
                electronic_type=ElectronicType.INSULATOR,
    
)

We can inspect the inputs parameters, which will be used to create the *pw.x* input file. The parameters are stored in an AiiDA Dictionary, which can be dumped to a python dictionary using the `get_dict` method:

In [16]:
builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 60.0,
  'ecutrho': 480.0},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

### (1.1) Change the pseudos to Norm-Conserving

We change the pseudo family to be the [PseudoDojo](http://www.pseudo-dojo.org/) one. 
The default family is the [SSSP](https://www.materialscloud.org/discover/sssp/table/efficiency) one, but here we need only Norm-Conserving pseudopotentials.

In [17]:
family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 

## (1.2) Modification of default inputs

It is possible to modify in two ways the inputs:

    (1) to use the overrides in the builder creation;
    (2) to re-define the input variables.

### Overrides

Overrides essentially act during the builder instance creation and, as the name suggests, ovverride the protocols inputs.

In [18]:
# (1):

overrides={
    'pw':{
        'parameters':{
            'SYSTEM': {
            'nosym': False,
            'occupations': 'fixed',
            'ecutwfc': 70.0,
            'ecutrho': 70*4,
            'force_symmorphic':True,
                      },
    },},}

builder = PwBaseWorkchain.get_builder_from_protocol(
                code=orm.load_code('pw-7.1@hydralogin'),
                structure=orm.load_node(161),
                protocol='fast',
                overrides=overrides,
                electronic_type=ElectronicType.INSULATOR,
                pseudo_family="PseudoDojo/0.4/PBE/SR/standard/upf",
    
)

family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 

builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic': True},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'conv_thr': 1.6e-09}}

### Re-define input parameters

The second way is to overwrite the inputs a posteriori, i.e. after the builder instance is created.

In [19]:
# (2)

pw_parameters = {
  'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.002,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic':True,},
 'ELECTRONS': {'electron_maxstep': 90, 'mixing_beta': 0.4, 'conv_thr': 8e-10},
}

builder.pw.parameters = orm.Dict(dict=pw_parameters)

builder.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'scf',
  'forc_conv_thr': 0.002,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0002},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'ecutwfc': 70.0,
  'ecutrho': 280,
  'force_symmorphic': True},
 'ELECTRONS': {'electron_maxstep': 90, 'mixing_beta': 0.4, 'conv_thr': 8e-10}}

Within protocols, the kpoints mesh is automatically choosen, by means of a `kpoints_distance` (inverse of the k-points density in the reciprocal space):

In [23]:
builder.kpoints_distance.value

0.5

We can defined our desired k-points mesh, by creating an istance of `KpointsData` and then setting the `builder.kpoints` attribute again:

In [24]:
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([4,4,4])

builder.kpoints = kpoints

Also, we set the workchain input `clean_workdir` to `False`, so we do not delete the remote folder after the simulation is finished (we need the data for the nscf step):

In [25]:
builder.clean_workdir = orm.Bool(False)

## (1.3) Setting the computational resources
 
Then you should set up the options about resources and submission settings. 
These have to be stored as a python dictionary in the metadata method:

In [26]:
builder.pw.metadata.options = {
    'max_wallclock_seconds': 60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

## (2) Run

Here we submit the simulation to the AiiDA daemon via the `submit` function. 

In [33]:
from aiida.engine import submit

In [28]:
run_scf = submit(builder)

We can see that the `run_scf` variable is indeed our `PwBaseWorkchain` just submitted:

In [37]:
print("PwBaseWorkchain instance:", run_scf)
print("PwBaseWorkchain pk:", run_scf.pk)
print("PwBaseWorkchain uuid:", run_scf.uuid)

PwBaseWorkchain instance: uuid: ac97fe6d-ae91-4072-aa19-71fe3436344f (pk: 1983) (aiida.workflows:quantumespresso.pw.base)
PwBaseWorkchain pk: 1983
PwBaseWorkchain uuid: ac97fe6d-ae91-4072-aa19-71fe3436344f


You can control the calculation from shell, typing one or more of these commands:

```bash
    verdi process list
    verdi process show <pk of the run_scf>
    verdi process report <pk of the run_scf>
```

It is possible to check if the calculation `is_finished_ok`:

In [38]:
run_scf.is_finished_ok

True

If this is the case, we can start inspecting the outputs of the calculation.

In [39]:
run_scf.outputs.output_parameters.get_dict()   #use tab to see available outputs

{'lsda': False,
 'energy': -729.59295119727,
 'volume': 34.754548090754,
 'fft_grid': [25, 25, 72],
 'energy_xc': -236.50857084077,
 'wall_time': '      4.60s ',
 'rho_cutoff': 3809.593683084,
 'symmetries': [{'t_rev': '0', 'symmetry_number': 0},
  {'t_rev': '0', 'symmetry_number': 2},
  {'t_rev': '0', 'symmetry_number': 26},
  {'t_rev': '0', 'symmetry_number': 27},
  {'t_rev': '0', 'symmetry_number': 28},
  {'t_rev': '0', 'symmetry_number': 29},
  {'t_rev': '0', 'symmetry_number': 33},
  {'t_rev': '0', 'symmetry_number': 35},
  {'t_rev': '0', 'symmetry_number': 56},
  {'t_rev': '0', 'symmetry_number': 57},
  {'t_rev': '0', 'symmetry_number': 62},
  {'t_rev': '0', 'symmetry_number': 63}],
 'wfc_cutoff': 952.398420771,
 'format_name': 'QEXSD',
 'lkpoint_dir': False,
 'occupations': 'fixed',
 'total_force': 0.0,
 'creator_name': 'pwscf',
 'energy_ewald': -516.99706371224,
 'energy_units': 'eV',
 'fermi_energy': 4.7464524860223,
 'forces_units': 'ev / angstrom',
 'q_real_space': False,
 '

## (3) Run the NSCF step

In [45]:
overrides={
    'pw':{
        'parameters':{
            'CONTROL': {
                'calculation': 'nscf',
                'restart_mode':'restart',},
            'SYSTEM': {
            'nosym': False,
            'occupations': 'fixed',
            'ecutwfc': 70.0,
            'ecutrho': 70*4,
            'nbnd':200,
            'force_symmorphic':True,
                      },
    },},}

builder = PwBaseWorkchain.get_builder_from_protocol(
    
                code=orm.load_code('pw-7.1@hydralogin'),
                structure=orm.load_node(161),
                protocol='fast',
                overrides=overrides,
                electronic_type=ElectronicType.INSULATOR,
                pseudo_family="PseudoDojo/0.4/PBE/SR/standard/upf",
    
)

#If you want to change the pseudos:
family = orm.load_group("PseudoDojo/0.4/PBE/SR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.pw.pseudos = family.get_pseudos(structure=orm.load_node(161)) 

builder.pw.metadata.options = {
    'max_wallclock_seconds': 60*60, # in seconds
    'resources': {
            "num_machines": 1, # nodes
            "num_mpiprocs_per_machine": 16, # MPI per nodes
            "num_cores_per_mpiproc": 1, # OPENMP
        },
    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed
    #'account':'project_name',
    'queue_name':'s3par',
    #'qos':'',
}

builder.clean_workdir = orm.Bool(False)

### Setting the parent scf calculation as starting point for the nscf.

We set as parent_folder the previous completed scf run. 
This needs to be provided by means of the `builder.pw.parent_folder` attribute.

In [46]:
parent_scf = orm.load_node(1983)
builder.pw.parent_folder = parent_scf.outputs.remote_folder

In [47]:
run_nscf = submit(builder)

In [51]:
run_nscf.pk

2014

In [54]:
run_nscf.is_finished_ok

False